In [1]:
import asyncio
import pandas as pd
from datetime import datetime, time, timedelta
from zoneinfo import ZoneInfo
from alpaca.data.live.stock import StockDataStream
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest, GetCalendarRequest
from alpaca.trading.enums import OrderSide, TimeInForce
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.enums import Adjustment

from IPython.display import clear_output


# Importing the modules themselves for reloading
import TradePosition
import TouchArea
import LiveTrader

# Reloading the modules to apply any changes
import importlib
importlib.reload(TradePosition)
importlib.reload(TouchArea)
importlib.reload(LiveTrader)


from TradePosition import TradePosition, SubPosition, export_trades_to_csv
from TouchArea import TouchArea, TouchAreaCollection
from LiveTrader import LiveTrader

import os, toml
from dotenv import load_dotenv

STANDARD_DATETIME_STR = '%Y-%m-%d %H:%M:%S'
ROUNDING_DECIMAL_PLACES = 10  # Choose an appropriate number of decimal places

load_dotenv(override=True)
livepaper = os.getenv('LIVEPAPER')
config = toml.load('../config.toml')

# Replace with your Alpaca API credentials
API_KEY = config[livepaper]['key']
API_SECRET = config[livepaper]['secret']


debug = True

def debug_print(*args, **kwargs):
    if debug:
        print(*args, **kwargs)

In [2]:
async def main():
    symbol = "AAPL"
    initial_balance = 10000
    trader = LiveTrader(API_KEY, API_SECRET, symbol, initial_balance)
    
    timeout = 70 # should be more than 60
    
    try:
        # Wait for up to 2 minutes (120 seconds) to receive data
        await asyncio.wait_for(trader.run(), timeout=timeout)
    except asyncio.TimeoutError:
        print(f"No data received within {timeout} seconds timeout. This is normal outside of market hours for IEX feed. \nNeed market data subcription to access SIP feed when outside market hours.")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        await trader.close()
        print("Trader stopped.")

In [3]:
import logging
logging.getLogger('alpaca').setLevel(logging.DEBUG)


import asyncio
import signal
from contextlib import asynccontextmanager

@asynccontextmanager
async def graceful_shutdown(trader):
    loop = asyncio.get_running_loop()
    try:
        yield
    finally:
        trader.log("Shutting down gracefully...")
        await trader.close_positions()
        await trader.close()
        pending = asyncio.all_tasks() - {asyncio.current_task()}
        for task in pending:
            task.cancel()
        await asyncio.gather(*pending, return_exceptions=True)
        loop.stop()

def signal_handler(trader):
    def handler(signum, frame):
        trader.log(f"Received signal {signum}. Initiating shutdown...")
        asyncio.create_task(trader.close())
    return handler

async def run_trader(trader: LiveTrader, timeout):
    try:
        await asyncio.wait_for(trader.run(), timeout=timeout)
    except asyncio.TimeoutError:
        trader.log(f"No data received within {timeout} seconds timeout. This is normal outside of market hours for IEX feed.")
    except Exception as e:
        trader.log(f"An error occurred in the trader: {e}", logging.ERROR)
    finally:
        await trader.close()
        print("Trader stopped.")

async def main():
    symbol = "AAPL"
    initial_balance = 10000
    timeout = 5  # 1 hour timeout, adjust as needed
    
    print('asdf')

    trader = LiveTrader(API_KEY, API_SECRET, symbol, initial_balance)
    print('asdf')
    loop = asyncio.get_running_loop()
    print('asdf')
    for sig in (signal.SIGINT, signal.SIGTERM):
        loop.add_signal_handler(sig, signal_handler(trader))
    print('asdf')
    async with graceful_shutdown(trader):
        await run_trader(trader, timeout)
    print('asdf')

In [4]:
import tracemalloc
tracemalloc.start()
import logging
logging.getLogger('alpaca').setLevel(logging.DEBUG)

# Run the main function
clear_output(wait=True)  # Clear previous output

# Check if we're already in an event loop
try:
    loop = asyncio.get_running_loop()
except RuntimeError:
    # If we're not in an event loop, create one and run our main function
    asyncio.run(main())
else:
    # If we're already in an event loop, create a task
    loop.create_task(main())

asdf
asdf
asdf
